# Airavata Experiment SDK - Molecular Dynamics Example

This SDK allows users to define, plan, and execute molecular dynamics experiments with ease.
Here we demonstrate how to authenticate, set up a NAMD experiment, add replicas, create an execution plan, and monitor the execution.

## Install the required packages

First, install the `airavata-python-sdk-test` package from the pip repository.

In [ ]:
%pip install --upgrade airavata-python-sdk-test

Obtaining file:///Users/yasith/projects/artisan/airavata/airavata-api/airavata-client-sdks/airavata-python-sdk
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for airavata-python-sdk-test (pyproject.toml) ... done
  Created wheel for airavata-python-sdk-test: filename=airavata_python_sdk_test-0.0.2-0.editable-py3-none-any.whl size=11284 sha256=c3d58cfa6d1cd393fa9ff8ff597e416e466721170a37adcd4b3429d39076dc3e
  Stored in directory: /private/var/folders/_n/fcf6nx4j67gbbt4_8mjqxdc80000gn/T/pip-ephem-wheel-cache-srerellb/wheels/6a/64/3a/ba5bbd28958f1b9f1f2d15d2c8999c899e17c402760ebd7d24
Successfully built airavata-python-sdk-test
  Attempting uninstall: airavata-python-sdk-test
    Found existing installation: airavata-python-sdk-test 0.0.2
    Uninstalling airavata-python-sdk-test-0.0.2:
      Successfully uninst

## Import the Experiments SDK

In [2]:
import airavata_experiments as ae
from airavata_experiments import md

## Authenticate for Remote Execution

To authenticate for remote execution, call the `ae.login()` method.
This method will prompt you to enter your credentials and authenticate your session.

In [3]:
ae.login()

Using saved token


Once authenticated, the `ae.list_runtimes()` function can be called to list HPC resources that the user can access.

In [ ]:
runtimes = ae.list_runtimes()
display(runtimes)

## Upload Experiment Files

Drag and drop experiment files onto the workspace that this notebook is run on.

```bash
(sh) $: tree data
data
├── b4pull.pdb
├── b4pull.restart.coor
├── b4pull.restart.vel
├── b4pull.restart.xsc
├── par_all36_water.prm
├── par_all36m_prot.prm
├── pull.conf
├── structure.pdb
└── structure.psf

1 directory, 9 files

```

## Define a NAMD Experiment

The `md.NAMD.initialize()` is used to define a NAMD experiment.
Here, provide the paths to the `.conf` file, the `.pdb` file, the `.psf` file, any optional files you want to run NAMD on.
You can preview the function definition through auto-completion.

```python
def initialize(
    name: str,
    config_file: str,
    pdb_file: str,
    psf_file: str,
    ffp_files: list[str],
    other_files: list[str] = [],
    parallelism: Literal['CPU', 'GPU'] = "CPU",
    num_replicas: int = 1
) -> Experiment[ExperimentApp]
```

In [ ]:
exp = md.NAMD.initialize(
    name="yasith_namd_experiment",
    config_file="data/pull.conf",
    pdb_file="data/structure.pdb",
    psf_file="data/structure.psf",
    ffp_files=[
      "data/par_all36_water.prm",
      "data/par_all36m_prot.prm"
    ],
    other_files=[
      "data/b4pull.pdb",
      "data/b4pull.restart.coor",
      "data/b4pull.restart.vel",
      "data/b4pull.restart.xsc",
    ],
    parallelism="GPU",
)

To add replica runs, simply call the `exp.add_replica()` function.
You can call the `add_replica()` function as many times as you want replicas.
Any optional resource constraint can be provided here.

In [ ]:
exp.add_replica()

## Create Execution Plan

Call the `exp.plan()` function to transform the experiment definition + replicas into a stateful execution plan.
This plan can be exported in JSON format and imported back.

In [ ]:
plan = exp.plan()  # this will create a plan for the experiment
plan.describe()  # this will describe the plan
plan.save_json("plan.json")  # save the plan state

## Execute the Plan

In [ ]:
plan = ae.load_plan("plan.json")
plan.launch()
plan.save_json("plan.json")  # save the plan state

## Option A - Wait for Completion

In [ ]:
plan = ae.load_plan("plan.json")
plan.describe()

In [7]:
plan = ae.load_plan("plan.json")
plan.join()

Output()

Interrupted by user.


## Option B - Terminate Execution

In [ ]:
plan = ae.load_plan("plan.json")
plan.stop()

## Option C - Monitor Files During Execution

Displaying the status and files generated by each replica (task)

In [6]:
plan = ae.load_plan("plan.json")
from pprint import pprint
for task in plan.tasks:
    status = task.status()
    print(status)
    files = task.files()
    pprint(files)

display(plan.tasks[0].cat("NAMD.stdout"))

ExperimentStatus(state=4, timeOfStateChange=1733417291473, reason='process  started', statusId='EXPERIMENT_STATE_451b84f8-b6e8-472c-84b6-23460a4ecbdf')
['1',
 'A1497186742',
 'NAMD.stderr',
 'NAMD.stdout',
 'NAMD_Repl_.dcd',
 'NAMD_Repl_1.out',
 'b4pull.pdb',
 'b4pull.restart.coor',
 'b4pull.restart.vel',
 'b4pull.restart.xsc',
 'job_1017394371.slurm',
 'par_all36_water.prm',
 'par_all36m_prot.prm',
 'pull.conf',
 'structure.pdb',
 'structure.psf',
 '']


"ExeTyp=GPU\nPJobID=\nrep_list=\nnum_rep=1\ninput=pull.conf\nagent_id=e016b89f-5eef-4e8e-b4eb-d202942dc76d\nserver_url=api.gateway.cybershuttle.org\n The Airavata Gateway User is scigap\n Namd run will use the input pull.conf\nNo replica array \nGPU executable will be used\nLoading gpu modules\nGPU Run Command is time -p mpirun --hostfile ./HostFile -np 10 namd3 +p10 pull.conf\nlrwxrwxrwx 1 scigap ind123 103 Dec  5 08:48 35590436 -> /expanse/lustre/scratch/scigap/temp_project/neuro-workdirs/PROCESS_55958093-5bfc-4f02-9a07-2e1086fe55eb\n Checking if replica run(s)\nreplar is \nTesting if replicas defined\n/expanse/lustre/scratch/scigap/temp_project/neuro-workdirs/PROCESS_55958093-5bfc-4f02-9a07-2e1086fe55eb/1\nstdout:  * Serving Flask app 'kernel'\nstdout:  * Debug mode: off\nstderr: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.\nstderr:  * Running on http://127.0.0.1:15000\nstderr: Press CTRL+C to quit\nCreating n

Displaying the intermediate results generated by each replica (task)

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

for index, task in enumerate(plan.tasks):

    @cs.task_context(task)
    def visualize():
        data = pd.read_csv("data.csv")
        plt.figure(figsize=(8, 6))
        plt.plot(data["x"], data["y"], marker="o", linestyle="-", linewidth=2, markersize=6)
        plt.title(f"Plot for Replica {index} of {len(plan.tasks)}")

    visualize()